In [4]:
# Import Libraries
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim

from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve , auc

from PIL import Image
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [5]:
# Specify transforms using torchvision.transforms as transforms
transformations = transforms.Compose([
    transforms.Resize(255),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [6]:
# Load in each dataset and apply transformations using
# the torchvision.datasets as datasets library
train_set = datasets.ImageFolder("CT_Skull_Fractures/data/train", transform = transformations)
val_set = datasets.ImageFolder("CT_Skull_Fractures/data/test", transform = transformations)

In [7]:
# Put into a Dataloader using torch library
train_loader = torch.utils.data.DataLoader(train_set, batch_size=32, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_set, batch_size =32, shuffle=True)

In [8]:
# Get pretrained model using torchvision.models as models library
model = models.densenet161(pretrained=True)

# Turn off training for their parameters
for param in model.parameters():
    param.requires_grad = False

In [9]:
# Create new classifier for model using torch.nn as nn library

# Initialize classifier
classifier_input = model.classifier.in_features

# number of classes
num_labels = 2 

classifier = nn.Sequential(nn.Linear(classifier_input, 1024),
                           nn.ReLU(),
                           nn.Linear(1024, 512),
                           nn.ReLU(),
                           nn.Linear(512, num_labels),
                           nn.LogSoftmax(dim=1))

# Replace default classifier with new classifier
model.classifier = classifier

In [10]:
# Find the device available to use using torch library
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move model to the device specified above
model.to(device)

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 96, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(96, 192, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(192, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(144, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (rel

In [11]:
# Set the error function using torch.nn as nn library
criterion = nn.NLLLoss()

# Set the optimizer function using torch.optim as optim library
optimizer = optim.Adam(model.classifier.parameters())

In [12]:
def train_net(epochs):
    lossiloss = []
    valiloss = []
    acc = []
    epoc = []
    tmp=[]
    o1 = []
    o2 = []
    for epoch in range(1,epochs+1):
        train_loss = 0
        val_loss = 0
        accuracy = 0

        # Training the model
        model.train()
        counter = 0
        print("\n\nEpoch ", epoch," - Training\n=====================")
        for inputs, labels in train_loader:
            # Move to device
            inputs, labels = inputs.to(device), labels.to(device)        # Clear optimizers
            optimizer.zero_grad()        # Forward pass
            output = model.forward(inputs)        # Loss
            loss = criterion(output, labels)        # Calculate gradients (backpropogation)
            loss.backward()        # Adjust parameters based on gradients
            optimizer.step()        # Add the loss to the training set's rnning loss
            train_loss += loss.item()*inputs.size(0)

            # Print the progress of our training
            counter += 1
            #print("Epoch ", epoch," - Training")
            sys.stdout.write(f"Step {counter:1d} / {len(train_loader):1d}\r")
            sys.stdout.flush()
            #sys.stdout.write(f"Fitting {clf_amount:1d}/{len(alphas)*n_trials:1d} Multi-Layer-Perceptron classifiers\r")
        # Evaluating the model
        model.eval()
        counter = 0
        print("Epoch ", epoch," - Validation\n=====================")
        # Tell torch not to calculate gradients
        with torch.no_grad():
            for inputs, labels in val_loader:
                # Move to device
                inputs, labels = inputs.to(device), labels.to(device)            # Forward pass
                output = model.forward(inputs)            # Calculate Loss
                valloss = criterion(output, labels)            # Add loss to the validation set's running loss
                val_loss += valloss.item()*inputs.size(0)
                
                tmp.append(val_loss)
                
                # Since our model outputs a LogSoftmax, find the real 
                # percentages by reversing the log function
                output = torch.exp(output)            # Get the top class of the output
                top_p, top_class = output.topk(1, dim=1)            # See how many of the classes were correct?
                equals = top_class == labels.view(*top_class.shape)            # Calculate the mean (get the accuracy for this batch)
                # and add it to the running accuracy for this epoch
                accuracy += torch.mean(equals.type(torch.FloatTensor)).item()

                # Print the progress of our evaluation
                counter += 1
                #print("Epoch ", epoch,"\n===============\nValidation\n===============\n")
                sys.stdout.write(f"Step {counter:1d} / {len(val_loader):1d}\r")
                sys.stdout.flush()  
        #print(tmp)
        # Get the average loss for the entire epoch
        train_loss = train_loss/len(train_loader.dataset)
        valid_loss = val_loss/len(val_loader.dataset)    
        ac = accuracy/len(val_loader)
        # Print out the information
        print('Accuracy: ', ac) # accuracy on val set
        print('Training Loss: {:.6f} \nValidation Loss: {:.6f}'.format(train_loss, valid_loss),'\n')

        lossiloss.append(train_loss)
        valiloss.append(valid_loss)
        acc.append(ac)
        epoc.append(epoch)
        
    return lossiloss , valiloss , acc , epoc

In [13]:
# Loss, Val Loss, Accuracy, number of epochs
lossiloss , valiloss , acc , epoc = train_net(epochs = 5) # change here the number of epochs



Epoch  1  - Training
Epoch  1  - Validation
Accuracy:  0.8335886439856361
Training Loss: 0.344143 
Validation Loss: 0.362548 



Epoch  2  - Training
Epoch  2  - Validation
Accuracy:  0.7966537309043548
Training Loss: 0.281208 
Validation Loss: 0.445404 



Epoch  3  - Training
Epoch  3  - Validation
Accuracy:  0.8655535131692886
Training Loss: 0.251624 
Validation Loss: 0.320828 



Epoch  4  - Training
Epoch  4  - Validation
Accuracy:  0.8249761707642499
Training Loss: 0.256623 
Validation Loss: 0.381662 



Epoch  5  - Training
Epoch  5  - Validation
Accuracy:  0.8686853217727998
Training Loss: 0.226523 
Validation Loss: 0.309296 



In [14]:
# Print results
print(lossiloss)
print()
print(valiloss)
print()
print(acc)
print()
print(epoc)

'\n# Print results\nprint(lossiloss)\nprint()\nprint(valiloss)\nprint()\nprint(acc)\nprint()\nprint(epoc)\n'

In [15]:
model.eval()

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 96, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(96, 192, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(192, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(144, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (rel

In [16]:
# save model for future fast inference

PATH = 'CT_Skull_Fractures/interactive/model_outputs/mymodel.pth'
torch.save(model.state_dict(), PATH)
print("model saved successfuly!")

# Loss, Val Loss, Accuracy, number of epochs
np.save("CT_Skull_Fractures/interactive/model_outputs/lossiloss.npy" ,  lossiloss)
np.save("CT_Skull_Fractures/interactive/model_outputs/valiloss.npy" , valiloss)
np.save("CT_Skull_Fractures/interactive/model_outputs/acc.npy" , acc)
np.save("CT_Skull_Fractures/interactive/model_outputs/epoc.npy" , epoc)

model saved successfuly!
